## एम्बेडिंग्स

हमारे पिछले उदाहरण में, हमने `vocab_size` लंबाई वाले उच्च-आयामी बैग-ऑफ-वर्ड्स वेक्टर पर काम किया था, और हमने निम्न-आयामी पोज़िशनल रिप्रेज़ेंटेशन वेक्टर को स्पष्ट रूप से स्पार्स वन-हॉट रिप्रेज़ेंटेशन में परिवर्तित किया था। यह वन-हॉट रिप्रेज़ेंटेशन मेमोरी-कुशल नहीं है। इसके अलावा, प्रत्येक शब्द को एक-दूसरे से स्वतंत्र रूप से माना जाता है, इसलिए वन-हॉट एन्कोडेड वेक्टर शब्दों के बीच के अर्थपूर्ण समानताओं को व्यक्त नहीं करते हैं।

इस यूनिट में, हम **News AG** डेटासेट का और अधिक अन्वेषण करेंगे। शुरू करने के लिए, आइए डेटा लोड करें और पिछले यूनिट से कुछ परिभाषाएँ प्राप्त करें।


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### एम्बेडिंग क्या है?

**एम्बेडिंग** का विचार यह है कि शब्दों को निम्न-आयामी घने वेक्टरों के रूप में प्रस्तुत किया जाए, जो शब्द के अर्थपूर्ण अर्थ को प्रतिबिंबित करते हैं। हम बाद में चर्चा करेंगे कि अर्थपूर्ण शब्द एम्बेडिंग कैसे बनाई जाए, लेकिन फिलहाल, एम्बेडिंग को शब्द वेक्टर की आयामीयता को कम करने के एक तरीके के रूप में सोचें। 

इस प्रकार, एक एम्बेडिंग लेयर एक शब्द को इनपुट के रूप में लेती है और निर्दिष्ट `embedding_size` का आउटपुट वेक्टर उत्पन्न करती है। एक तरह से, यह `Dense` लेयर के समान है, लेकिन यह एक-हॉट एन्कोडेड वेक्टर को इनपुट के रूप में लेने के बजाय, शब्द संख्या को इनपुट के रूप में ले सकती है।

हमारे नेटवर्क में पहली लेयर के रूप में एम्बेडिंग लेयर का उपयोग करके, हम बैग-ऑफ-वर्ड्स से **एम्बेडिंग बैग** मॉडल में स्विच कर सकते हैं, जहां हम पहले अपने टेक्स्ट के प्रत्येक शब्द को संबंधित एम्बेडिंग में परिवर्तित करते हैं, और फिर उन सभी एम्बेडिंग पर कुछ समग्र फ़ंक्शन की गणना करते हैं, जैसे `sum`, `average` या `max`।  

![पांच अनुक्रम शब्दों के लिए एक एम्बेडिंग क्लासिफायर दिखाने वाली छवि।](../../../../../lessons/5-NLP/14-Embeddings/images/embedding-classifier-example.png)

हमारे क्लासिफायर न्यूरल नेटवर्क में निम्नलिखित लेयर शामिल हैं:

* `TextVectorization` लेयर, जो एक स्ट्रिंग को इनपुट के रूप में लेती है और टोकन नंबरों का एक टेन्सर उत्पन्न करती है। हम एक उचित शब्दावली आकार `vocab_size` निर्दिष्ट करेंगे और कम बार उपयोग किए जाने वाले शब्दों को अनदेखा करेंगे। इनपुट आकार 1 होगा, और आउटपुट आकार $n$ होगा, क्योंकि हमें $n$ टोकन प्राप्त होंगे, जिनमें से प्रत्येक में 0 से `vocab_size` तक की संख्या होगी।
* `Embedding` लेयर, जो $n$ नंबर लेती है और प्रत्येक नंबर को एक निर्दिष्ट लंबाई (हमारे उदाहरण में 100) के घने वेक्टर में बदल देती है। इस प्रकार, $n$ आकार के इनपुट टेन्सर को $n\times 100$ आकार के टेन्सर में परिवर्तित किया जाएगा। 
* एग्रीगेशन लेयर, जो इस टेन्सर का पहले अक्ष के साथ औसत लेती है, यानी यह विभिन्न शब्दों से संबंधित सभी $n$ इनपुट टेन्सर का औसत गणना करेगी। इस लेयर को लागू करने के लिए, हम एक `Lambda` लेयर का उपयोग करेंगे और उसमें औसत गणना करने के लिए फ़ंक्शन पास करेंगे। आउटपुट का आकार 100 होगा, और यह पूरे इनपुट अनुक्रम का संख्यात्मक प्रतिनिधित्व होगा।
* अंतिम `Dense` रैखिक क्लासिफायर।


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


`summary` प्रिंटआउट में, **output shape** कॉलम में पहला टेंसर डायमेंशन `None` मिनीबैच साइज को दर्शाता है, और दूसरा टोकन अनुक्रम की लंबाई को। मिनीबैच में सभी टोकन अनुक्रमों की लंबाई अलग-अलग होती है। हम अगले सेक्शन में इसे संभालने के तरीके पर चर्चा करेंगे।

अब चलिए नेटवर्क को ट्रेन करते हैं:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **नोट** कि हम डेटा के एक उपसमुच्चय के आधार पर वेक्टराइज़र बना रहे हैं। यह प्रक्रिया को तेज करने के लिए किया जाता है, और इससे ऐसी स्थिति उत्पन्न हो सकती है जब हमारे पाठ के सभी टोकन शब्दावली में मौजूद न हों। इस स्थिति में, उन टोकनों को अनदेखा कर दिया जाएगा, जिससे सटीकता में थोड़ी कमी हो सकती है। हालांकि, वास्तविक जीवन में पाठ का एक उपसमुच्चय अक्सर शब्दावली का अच्छा अनुमान प्रदान करता है।


### परिवर्तनीय अनुक्रम आकारों से निपटना

आइए समझते हैं कि मिनीबैच में प्रशिक्षण कैसे होता है। ऊपर दिए गए उदाहरण में, इनपुट टेन्सर का आयाम 1 है, और हम 128-लंबे मिनीबैच का उपयोग करते हैं, जिससे टेन्सर का वास्तविक आकार $128 \times 1$ हो जाता है। हालांकि, प्रत्येक वाक्य में टोकन की संख्या अलग-अलग होती है। यदि हम `TextVectorization` लेयर को एकल इनपुट पर लागू करते हैं, तो लौटाए गए टोकन की संख्या अलग-अलग होती है, यह इस बात पर निर्भर करता है कि टेक्स्ट को कैसे टोकनाइज़ किया गया है:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


हालांकि, जब हम वेक्टराइज़र को कई अनुक्रमों पर लागू करते हैं, तो इसे आयताकार आकार का एक टेंसर उत्पन्न करना होता है, इसलिए यह अप्रयुक्त तत्वों को PAD टोकन (जो हमारे मामले में शून्य है) से भरता है:


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

यहां हम एम्बेडिंग्स देख सकते हैं:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **नोट**: पैडिंग की मात्रा को कम करने के लिए, कुछ मामलों में यह समझदारी होती है कि डेटासेट में सभी अनुक्रमों को उनकी लंबाई बढ़ने के क्रम में (या अधिक सटीक रूप से, टोकन की संख्या के अनुसार) क्रमबद्ध किया जाए। इससे यह सुनिश्चित होगा कि प्रत्येक मिनीबैच में समान लंबाई के अनुक्रम शामिल हों।


## सेमांटिक एम्बेडिंग्स: वर्ड2वेक

हमारे पिछले उदाहरण में, एम्बेडिंग लेयर ने शब्दों को वेक्टर प्रतिनिधित्व में मैप करना सीखा, लेकिन इन प्रतिनिधित्वों में सेमांटिक अर्थ नहीं था। यह अच्छा होगा कि हम एक ऐसा वेक्टर प्रतिनिधित्व सीखें जिसमें समान शब्द या पर्यायवाची शब्द कुछ वेक्टर दूरी (जैसे यूक्लिडियन दूरी) के संदर्भ में एक-दूसरे के करीब हों।

इसके लिए, हमें अपने एम्बेडिंग मॉडल को [Word2Vec](https://en.wikipedia.org/wiki/Word2vec) जैसी तकनीक का उपयोग करके बड़े टेक्स्ट संग्रह पर प्रीट्रेन करना होगा। यह दो मुख्य आर्किटेक्चर पर आधारित है जो शब्दों का वितरित प्रतिनिधित्व उत्पन्न करने के लिए उपयोग किए जाते हैं:

 - **कंटीन्युअस बैग-ऑफ-वर्ड्स** (CBoW), जिसमें हम मॉडल को आस-पास के संदर्भ से एक शब्द की भविष्यवाणी करने के लिए प्रशिक्षित करते हैं। दिए गए ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$ में, मॉडल का लक्ष्य $(W_{-2},W_{-1},W_1,W_2)$ से $W_0$ की भविष्यवाणी करना है।
 - **कंटीन्युअस स्किप-ग्राम** CBoW के विपरीत है। यह मॉडल संदर्भ शब्दों की आस-पास की विंडो का उपयोग करके वर्तमान शब्द की भविष्यवाणी करता है।

CBoW तेज है, जबकि स्किप-ग्राम धीमा है, लेकिन यह कम बार उपयोग होने वाले शब्दों का बेहतर प्रतिनिधित्व करता है।

![CBoW और स्किप-ग्राम एल्गोरिदम को शब्दों को वेक्टर में बदलने के लिए दिखाने वाली छवि।](../../../../../lessons/5-NLP/14-Embeddings/images/example-algorithms-for-converting-words-to-vectors.png)

Google News डेटासेट पर प्रीट्रेन किए गए Word2Vec एम्बेडिंग के साथ प्रयोग करने के लिए, हम **gensim** लाइब्रेरी का उपयोग कर सकते हैं। नीचे हम 'neural' के सबसे समान शब्दों को ढूंढते हैं।

> **Note:** जब आप पहली बार शब्द वेक्टर बनाते हैं, तो उन्हें डाउनलोड करने में कुछ समय लग सकता है!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


हम शब्द से वेक्टर एम्बेडिंग भी निकाल सकते हैं, जिसे वर्गीकरण मॉडल के प्रशिक्षण में उपयोग किया जा सकता है। एम्बेडिंग में 300 घटक होते हैं, लेकिन यहां स्पष्टता के लिए हम केवल वेक्टर के पहले 20 घटक दिखा रहे हैं:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

सार्थक एम्बेडिंग की महान बात यह है कि आप अर्थ के आधार पर वेक्टर एन्कोडिंग को संशोधित कर सकते हैं। उदाहरण के लिए, हम ऐसा शब्द खोजने के लिए कह सकते हैं जिसका वेक्टर प्रतिनिधित्व *राजा* और *महिला* शब्दों के जितना करीब हो सके, और *पुरुष* शब्द से जितना दूर हो सके:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

ऊपर दिए गए उदाहरण में कुछ आंतरिक GenSym जादू का उपयोग किया गया है, लेकिन मूल तर्क वास्तव में काफी सरल है। एम्बेडिंग्स के बारे में एक दिलचस्प बात यह है कि आप एम्बेडिंग वेक्टर पर सामान्य वेक्टर संचालन कर सकते हैं, और वह शब्दों के **अर्थों** पर संचालन को प्रतिबिंबित करेगा। ऊपर दिए गए उदाहरण को वेक्टर संचालन के रूप में व्यक्त किया जा सकता है: हम **KING-MAN+WOMAN** के अनुरूप वेक्टर की गणना करते हैं (संबंधित शब्दों के वेक्टर प्रतिनिधित्व पर `+` और `-` संचालन किए जाते हैं), और फिर उस वेक्टर के सबसे निकटतम शब्द को शब्दकोश में खोजते हैं:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOTE**: हमने *man* और *woman* वेक्टर में एक छोटा गुणांक जोड़ना पड़ा - इसे हटाकर देखें कि क्या होता है।

सबसे नज़दीकी वेक्टर खोजने के लिए, हम TensorFlow की तकनीक का उपयोग करते हैं ताकि हमारे वेक्टर और शब्दावली में सभी वेक्टर के बीच की दूरी का वेक्टर प्राप्त किया जा सके, और फिर `argmin` का उपयोग करके न्यूनतम शब्द का इंडेक्स खोजा जा सके।


हालांकि Word2Vec शब्दार्थ को व्यक्त करने का एक शानदार तरीका लगता है, इसके कई नुकसान भी हैं, जिनमें निम्नलिखित शामिल हैं:

* CBoW और skip-gram मॉडल दोनों **पूर्वानुमानात्मक एम्बेडिंग्स** हैं, और ये केवल स्थानीय संदर्भ को ध्यान में रखते हैं। Word2Vec वैश्विक संदर्भ का लाभ नहीं उठाता।
* Word2Vec शब्द की **रूप-रचना** (morphology) को ध्यान में नहीं रखता, यानी इस तथ्य को कि शब्द का अर्थ उसके विभिन्न भागों, जैसे मूल (root), पर निर्भर कर सकता है।  

**FastText** दूसरे प्रतिबंध को दूर करने की कोशिश करता है और Word2Vec पर आधारित होकर प्रत्येक शब्द और उसमें पाए जाने वाले अक्षर n-grams के लिए वेक्टर प्रतिनिधित्व सीखता है। इन प्रतिनिधित्वों के मानों को प्रत्येक प्रशिक्षण चरण में एक वेक्टर में औसतित किया जाता है। हालांकि यह पूर्व-प्रशिक्षण में अतिरिक्त गणना जोड़ता है, यह शब्द एम्बेडिंग्स को उप-शब्द जानकारी को एन्कोड करने में सक्षम बनाता है।

एक अन्य विधि, **GloVe**, शब्द एम्बेडिंग्स के लिए एक अलग दृष्टिकोण अपनाती है, जो शब्द-संदर्भ मैट्रिक्स के गुणनखंडन (factorization) पर आधारित है। सबसे पहले, यह एक बड़ा मैट्रिक्स बनाता है जो विभिन्न संदर्भों में शब्दों की घटनाओं की संख्या को गिनता है, और फिर यह इस मैट्रिक्स को निम्न आयामों में इस तरह से प्रस्तुत करने की कोशिश करता है कि पुनर्निर्माण हानि (reconstruction loss) न्यूनतम हो।

gensim लाइब्रेरी इन शब्द एम्बेडिंग्स का समर्थन करती है, और आप ऊपर दिए गए मॉडल लोडिंग कोड को बदलकर इनके साथ प्रयोग कर सकते हैं।


## Keras में प्रीट्रेंड एम्बेडिंग्स का उपयोग करना

हम ऊपर दिए गए उदाहरण को संशोधित कर सकते हैं ताकि हमारे एम्बेडिंग लेयर की मैट्रिक्स को वर्ड2वेक जैसे सेमांटिक एम्बेडिंग्स से पहले से भर सकें। प्रीट्रेंड एम्बेडिंग और टेक्स्ट कॉर्पस की शब्दावली संभवतः मेल नहीं खाएगी, इसलिए हमें एक को चुनना होगा। यहां हम दो संभावित विकल्पों का पता लगाते हैं: टोकनाइज़र शब्दावली का उपयोग करना, और वर्ड2वेक एम्बेडिंग्स की शब्दावली का उपयोग करना।

### टोकनाइज़र शब्दावली का उपयोग करना

जब टोकनाइज़र शब्दावली का उपयोग करते हैं, तो शब्दावली के कुछ शब्दों के लिए वर्ड2वेक एम्बेडिंग्स उपलब्ध होंगे, और कुछ गायब होंगे। मान लें कि हमारी शब्दावली का आकार `vocab_size` है, और वर्ड2वेक एम्बेडिंग वेक्टर की लंबाई `embed_size` है, तो एम्बेडिंग लेयर को `vocab_size`$\times$`embed_size` आकार की वेट मैट्रिक्स द्वारा दर्शाया जाएगा। हम इस मैट्रिक्स को शब्दावली के माध्यम से जाकर भरेंगे:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


शब्दों के लिए जो Word2Vec शब्दावली में मौजूद नहीं हैं, हम उन्हें शून्य के रूप में छोड़ सकते हैं, या एक रैंडम वेक्टर उत्पन्न कर सकते हैं।

अब हम प्रीट्रेंड वेट्स के साथ एक एम्बेडिंग लेयर को परिभाषित कर सकते हैं:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **नोट**: ध्यान दें कि जब हम `Embedding` बनाते समय `trainable=False` सेट करते हैं, तो इसका मतलब है कि हम Embedding लेयर को पुनः प्रशिक्षित नहीं कर रहे हैं। इससे सटीकता थोड़ी कम हो सकती है, लेकिन यह प्रशिक्षण को तेज कर देता है।

### एम्बेडिंग शब्दावली का उपयोग करना

पिछले दृष्टिकोण के साथ एक समस्या यह है कि TextVectorization और Embedding में उपयोग की गई शब्दावलियां अलग-अलग हैं। इस समस्या को हल करने के लिए, हम निम्नलिखित समाधानों में से एक का उपयोग कर सकते हैं:
* हमारे शब्दावली पर Word2Vec मॉडल को पुनः प्रशिक्षित करें।
* प्रीट्रेंड Word2Vec मॉडल की शब्दावली के साथ हमारा डेटासेट लोड करें। डेटासेट को लोड करते समय उपयोग की जाने वाली शब्दावलियां निर्दिष्ट की जा सकती हैं।

दूसरा दृष्टिकोण आसान लगता है, तो चलिए इसे लागू करते हैं। सबसे पहले, हम Word2Vec एम्बेडिंग से ली गई निर्दिष्ट शब्दावली के साथ एक `TextVectorization` लेयर बनाएंगे:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

जेनसिम वर्ड एम्बेडिंग्स लाइब्रेरी में एक सुविधाजनक फ़ंक्शन, `get_keras_embeddings`, होता है, जो आपके लिए स्वचालित रूप से संबंधित Keras एम्बेडिंग्स लेयर बना देगा।


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


हम जो उच्च सटीकता नहीं देख रहे हैं, उसके कारणों में से एक यह है कि हमारे डेटासेट के कुछ शब्द प्रीट्रेंड GloVe शब्दावली में नहीं हैं, और इसलिए उन्हें अनदेखा कर दिया जाता है। इसे दूर करने के लिए, हम अपने डेटासेट के आधार पर अपने स्वयं के एम्बेडिंग्स को प्रशिक्षित कर सकते हैं।


## संदर्भात्मक एम्बेडिंग

पारंपरिक प्रीट्रेंड एम्बेडिंग जैसे Word2Vec की एक मुख्य सीमा यह है कि, भले ही वे किसी शब्द का कुछ अर्थ पकड़ सकते हैं, वे विभिन्न अर्थों के बीच अंतर नहीं कर सकते। यह डाउनस्ट्रीम मॉडल्स में समस्याएं पैदा कर सकता है।

उदाहरण के लिए, शब्द 'play' का इन दो वाक्यों में अलग-अलग अर्थ है:
- मैं थिएटर में एक **play** देखने गया।
- जॉन अपने दोस्तों के साथ **play** करना चाहता है।

हमने जिन प्रीट्रेंड एम्बेडिंग की बात की, वे शब्द 'play' के दोनों अर्थों को एक ही एम्बेडिंग में दर्शाते हैं। इस सीमा को दूर करने के लिए, हमें **भाषा मॉडल** पर आधारित एम्बेडिंग बनानी होगी, जो बड़े टेक्स्ट कॉर्पस पर प्रशिक्षित होता है और *जानता है* कि शब्दों को विभिन्न संदर्भों में कैसे जोड़ा जा सकता है। संदर्भात्मक एम्बेडिंग पर चर्चा करना इस ट्यूटोरियल के दायरे से बाहर है, लेकिन हम अगले यूनिट में भाषा मॉडल्स पर चर्चा करते समय इस पर वापस आएंगे।



---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता सुनिश्चित करने का प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को प्रामाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।
